In [1]:
# Install the package
!pip install -U crawl4ai

# For pre release versions
!pip install crawl4ai --pre

# Run post-installation setup
!crawl4ai-setup

# Verify your installation
!crawl4ai-doctor!

[INIT].... → Running post-installation setup... 
[INIT].... → Installing Playwright browsers... 
Installing dependencies...
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Fetched 6,555 B in 2s (4,001 B/s)
Reading package lists... Done
W: Skipping acquire of configured f

In [1]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [3]:
!pip install accelerate bitsandbytes

In [ ]:
import asyncio
import torch
import json
import re # <--- Thêm thư viện này để xử lý chuỗi mạnh hơn
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# --- CẤU HÌNH MODEL (GIỮ NGUYÊN) ---
# MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"
# MODEL_ID = "Qwen/Qwen2.5-3B-Instruct"
MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"
_CACHED_MODEL = None
_CACHED_TOKENIZER = None

def load_local_llm():
    global _CACHED_MODEL, _CACHED_TOKENIZER
    if _CACHED_MODEL is not None:
        print("⚡ Model đã nằm sẵn trong GPU. Dùng lại luôn!")
        return _CACHED_TOKENIZER, _CACHED_MODEL

    print(f"⏳ Đang tải model {MODEL_ID} lần đầu...")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16
    )

    _CACHED_TOKENIZER = AutoTokenizer.from_pretrained(MODEL_ID)
    _CACHED_MODEL = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )
    print("✅ Đã nạp Model thành công!")
    return _CACHED_TOKENIZER, _CACHED_MODEL

# --- HÀM TRÍCH XUẤT (ĐÃ NÂNG CẤP LOGIC CẮT ĐUÔI) ---
def extract_info_with_transformers(tokenizer, model, markdown_content):
    # 1. Prompt: Yêu cầu trả về mảng (List) để dễ cắt gọt
    prompt = f"""
    Dưới đây là nội dung trang web:
    '''
    {markdown_content[:3000]}
    '''
    Nhiệm vụ: Trích xuất danh sách các workflow/sản phẩm.
    Định dạng bắt buộc: Một JSON List `[...]` chứa các object.
    JSON Output:
    """

    messages = [
        {"role": "system", "content": "Bạn là trợ lý dữ liệu. Chỉ trả về JSON List hợp lệ."},
        {"role": "user", "content": prompt}
    ]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # 2. Tăng token lên một chút cho thoải mái (1024)
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1024,
        temperature=0.1,
        do_sample=True
    )

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # --- PHẦN LOGIC MỚI: TỰ ĐỘNG SỬA JSON BỊ CẮT ---

    # Bước A: Làm sạch cơ bản
    if "JSON Output:" in response:
        json_str = response.split("JSON Output:")[-1].strip()
    else:
        json_str = response

    json_str = json_str.replace("```json", "").replace("```", "").strip()

    # Bước B: Kỹ thuật "Cứu vãn" JSON bị cắt (Truncated JSON Repair)
    # Nếu chuỗi không kết thúc bằng ']' (tức là bị hết token giữa chừng)
    if not json_str.endswith("]"):
        print("⚠️ Cảnh báo: JSON bị cắt do hết token. Đang tự động sửa...")

        # Tìm vị trí dấu đóng ngoặc nhọn '}' cuối cùng hợp lệ
        last_object_end = json_str.rfind("}")

        if last_object_end != -1:
            # Cắt bỏ phần thừa phía sau dấu } cuối cùng
            # Ví dụ: [{"a":1}, {"b": 2}, {"c": hihi...  -> [{"a":1}, {"b": 2}]
            json_str = json_str[:last_object_end+1]

            # Đóng ngoặc vuông lại cho thành List hoàn chỉnh
            json_str += "]"
        else:
            # Trường hợp xấu nhất: không có object nào hoàn chỉnh
            return "[]"

    return json_str


In [3]:

# --- MAIN (THÊM PHẦN LƯU FILE) ---
async def main():
    tokenizer, model = load_local_llm()

    browser_conf = BrowserConfig(headless=True)
    run_conf = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS,
        word_count_threshold=50
    )

    url = "https://n8n.io/workflows/categories/ai/"
    print(f"\n🕷️ Đang cào dữ liệu từ: {url}")

    async with AsyncWebCrawler(config=browser_conf) as crawler:
        result = await crawler.arun(url=url, config=run_conf)

        if result.success:
            print("📝 Đang xử lý AI (Tự động cắt nếu quá dài)...")
            ai_result = extract_info_with_transformers(tokenizer, model, result.markdown)

            print("\n=== KẾT QUẢ ĐÃ XỬ LÝ ===")
            print(ai_result[:500] + "...") # Chỉ in một phần để check

            # --- LƯU FILE JSON ---
            try:
                # Parse thử sang Python Object để chắc chắn nó đúng chuẩn
                data_obj = json.loads(ai_result)

                # Lưu xuống đĩa
                filename = "du_lieu_ai.json"
                with open(filename, "w", encoding="utf-8") as f:
                    json.dump(data_obj, f, ensure_ascii=False, indent=4)

                print(f"\n✅ Đã lưu thành công vào file: {filename}")
                print(f"   Tổng cộng lấy được: {len(data_obj)} items")

            except json.JSONDecodeError as e:
                print(f"\n❌ JSON vẫn lỗi dù đã sửa: {e}")
                # Vẫn lưu file text thô để debug
                with open("error_raw.txt", "w", encoding="utf-8") as f:
                    f.write(ai_result)
        else:
            print(f"Lỗi Crawl: {result.error_message}")

if __name__ == "__main__":
    # Nếu chạy trên Colab nhớ thêm nest_asyncio.apply() ở đầu
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main())

⏳ Đang tải model Qwen/Qwen2.5-7B-Instruct lần đầu...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Đã nạp Model thành công!

🕷️ Đang cào dữ liệu từ: https://n8n.io/workflows/categories/ai/


[INIT].... → Crawl4AI 0.8.0 

[FETCH]... ↓ https://n8n.io/workflows/categories/ai/                                                              |
✓ | ⏱: 4.39s 

[SCRAPE].. ◆ https://n8n.io/workflows/categories/ai/                                                              |
✓ | ⏱: 0.16s 

[COMPLETE] ● https://n8n.io/workflows/categories/ai/                                                              |
✓ | ⏱: 4.60s 

📝 Đang xử lý AI (Tự động cắt nếu quá dài)...

=== KẾT QUẢ ĐÃ XỬ LÝ ===
assistant

[
    {
        "name": "Angie, personal AI assistant with Telegram voice and text",
        "url": "https://n8n.io/workflows/2462-angie-personal-ai-assistant-with-telegram-voice-and-text/"
    },
    {
        "name": "Sticky Note",
        "url": "https://n8n.io/workflows/2462-angie-personal-ai-assistant-with-telegram-voice-and-text/"
    },
    {
        "name": "Basic LLM Chain",
        "url": "https://n8n.io/workflows/2462-angie-personal-ai-assistant-with-telegram-voice-and-text...

❌ JSON vẫn lỗi dù đã sửa: Expecting value: line 1 column 1 (char 0)
